In [19]:
import pandas as pd

In [29]:
%%time
df_groundtruth = pd.read_csv('TestCases_groundTruth.csv')

CPU times: total: 0 ns
Wall time: 15.8 ms


In [30]:
df_groundtruth.rename(columns = {'Unnamed: 0':'test_case_ID', 
                                 'Unnamed: 1':'phenotypes', 
                                 'Unnamed: 2':'disease_name', 
                                 'Test Cases':'disease'}, inplace = True)

In [31]:
df_groundtruth = pd.DataFrame(df_groundtruth,columns=['test_case_ID', 'phenotypes','disease_name', 'disease'])
df_groundtruth.fillna('', inplace=True)
df_groundtruth.drop([0,1],inplace=True)
df_groundtruth.reset_index(drop=True,inplace=True)
df_groundtruth.head(10)

,test_case_ID,phenotypes,disease_name,disease
0,Stavropoulos et al,,,
1,TCS001,"HP:0010301, HP:0000252,",Rubinstein-Taybi Syndrome 2,ORPHA353284 (Rubinstein-Taybi
2,,HP:0001263,,syndrome due to EP300
3,,,,haploinsufficiency)
4,TCS003,"HP:0001263, HP:0002650,",Coffin-Siris syndrome,ORPHA1465 (Coffin-Siris
5,,"HP:0001382, HP:0002079,",,syndrome)
6,,HP:0001999,,
7,TCS004,"HP:0001263, HP:0004322,",Alazami Syndrome,ORPHA319671 (Microcephalic
8,,HP:0100255,,"primordial dwarfism, Alazami type)"
9,TCS005,HP:0007105,KAT6B-Related Disorder,ORPHA85201 (Genitopatellar


In [32]:
def data_set_cleanup(row):
    data = {'phen':'',
            'disease_name':'',
            'disease_id':''
           }
    if row[0] == '':
        data['phen'] = row[1] 
        data['disease_name'] = row[2] 
        data['disease_id'] = row[3] 
#     print(data,row)
    return data

In [33]:
df_groundtruth['new_col'] = df_groundtruth.apply(data_set_cleanup, axis=1)
x = 0
while x < df_groundtruth.shape[0]:
    if df_groundtruth.at[x,'test_case_ID']=='':
            df_groundtruth.at[x-1,'phenotypes'] += df_groundtruth.at[x,'phenotypes']
            df_groundtruth.at[x-1,'disease_name'] += df_groundtruth.at[x,'disease_name']
            df_groundtruth.at[x-1,'disease'] += df_groundtruth.at[x,'disease']
            df_groundtruth.drop(x,inplace=True)
            df_groundtruth.reset_index(drop=True,inplace=True)
    else:
        x += 1
df_groundtruth =  df_groundtruth[df_groundtruth['test_case_ID'].str.startswith('TCS')]
df_groundtruth.drop(columns='new_col',inplace=True)
df_groundtruth.reset_index(drop=True,inplace=True)
df_groundtruth

,test_case_ID,phenotypes,disease_name,disease
0,TCS001,"HP:0010301, HP:0000252,HP:0001263",Rubinstein-Taybi Syndrome 2,ORPHA353284 (Rubinstein-Taybisyndrome due to E...
1,TCS003,"HP:0001263, HP:0002650,HP:0001382, HP:0002079,...",Coffin-Siris syndrome,ORPHA1465 (Coffin-Sirissyndrome)
2,TCS004,"HP:0001263, HP:0004322,HP:0100255",Alazami Syndrome,"ORPHA319671 (Microcephalicprimordial dwarfism,..."
3,TCS005,HP:0007105,KAT6B-Related Disorder,"ORPHA85201 (Genitopatellarsyndrome), ORPHA3047..."
4,TCS006,"HP:0009372, HP:0001193,HP:0007598, HP:0001762",Type C Brachydactyly,ORPHA93384 (Brachydactyly typeC)
...,...,...,...,...
224,TCS240,"HP:0005616, HP:0000646,HP:0001263, HP:0001999,...",autosomal dominantpseudohypoparathyroidismtype...,"OR:79443, OR:79444"
225,TCS241,"HP:0001263, HP:0001999,HP:0001510, HP:0001382,...",autosomal dominant SHORTsyndrome [MIM: 269880],OR:3163
226,TCS242,"HP:0000062, HP:0001328","autosomal dominant sex-limited 46,XY sex rever...","OR:242, OR:251510"
227,TCS243,"HP:0000365, HP:0000365,HP:0007703, HP:0000662,...",autosomal dominant retinitispigmentosa (type 3...,"OR:53540, OR:791"


In [35]:
import re
def fix(a):
    return int(a[1:])
def extract_oid(row):
    try:
        temp = re.findall(r'OR[A-Z:]{1,3}\d+', row[3])
        res = [int(re.findall(r'\d+', x)[0]) for x in temp ]
        return res
    except:
        pass

In [36]:
df_groundtruth[df_groundtruth['test_case_ID'] == 'TCS021']

,test_case_ID,phenotypes,disease_name,disease
16,TCS021,"HP:0001107, HP:0001249,HP:0001513, HP:0007018",Oculocutaneous albinism type1 and MC4R-linked ...,ORPHA352731 (Oculocutaneousalbinism type 1)


In [37]:
df_groundtruth['oid'] = df_groundtruth.apply(extract_oid,axis=1)
df_groundtruth['phenotype'] = df_groundtruth.apply(lambda row:[x.replace("'","").replace(' ','') for x in row[1].split(sep=',') if 'HP' in x ],axis=1)
df_groundtruth = pd.DataFrame(df_groundtruth,columns=['phenotype','oid'])
df_groundtruth

,phenotype,oid
0,"[HP:0010301, HP:0000252, HP:0001263]",[353284]
1,"[HP:0001263, HP:0002650, HP:0001382, HP:000207...",[1465]
2,"[HP:0001263, HP:0004322, HP:0100255]",[319671]
3,[HP:0007105],"[85201, 3047]"
4,"[HP:0009372, HP:0001193, HP:0007598, HP:0001762]",[93384]
...,...,...
224,"[HP:0005616, HP:0000646, HP:0001263, HP:000199...","[79443, 79444]"
225,"[HP:0001263, HP:0001999, HP:0001510, HP:000138...",[3163]
226,"[HP:0000062, HP:0001328]","[242, 251510]"
227,"[HP:0000365, HP:0000365, HP:0007703, HP:000066...","[53540, 791]"


In [39]:
df_groundtruth.loc[16]

phenotype    [HP:0001107, HP:0001249, HP:0001513, HP:0007018]
oid                                                  [352731]
Name: 16, dtype: object

In [41]:
df_groundtruth.to_csv('ground_truth.csv',index=False)

In [42]:
df_results = pd.read_csv('testcases_results_orpha_id.txt',sep=':')
df_results.loc[16]

phenotype          ['HP:0001107', ' HP:0001249']
oid           [96253, 59315, 1912, 1911, 231222]
Name: 16, dtype: object

In [14]:
def dataset_cleanup(row):
    lst1 = [x.replace("'","").replace(' ','') for x in row[0][1:-1].split(sep=',') if 'HP' in x ]
    lst2 = [x for x in row[1][2:-1].split(sep=',') ]
    return lst1,lst2

In [17]:
df_results['new_col'] = df_results.apply(dataset_cleanup, axis=1)
for x in range(df_results.shape[0]):
    df_results.at[x,'phenotype'] = df_results.at[x,'new_col'][0]
    df_results.at[x,'oid '] = df_results.at[x,'new_col'][1]
df_results.drop(columns='new_col',inplace=True)
df_results.rename(columns={'oid ':'oid'},inplace=True)
df_results.head(10)

,phenotype,oid
0,"[HP:0010301, HP:0000252, HP:0001263]","[349, 813, 542306, 2752, 3208]"
1,"[HP:0001263, HP:0002650, HP:0001382, HP:000207...","[349, 813, 542306, 2752, 3208]"
2,"[HP:0001263, HP:0004322, HP:0100255]","[349, 813, 542306, 2752, 3208]"
3,[HP:0007105],"[580, 254519, 96253, 59315, 1912]"
4,"[HP:0009372, HP:0001193, HP:0007598, HP:0001762]","[349, 503, 254361, 813, 397590]"
5,[HP:0001263],"[349, 813, 542306, 2752, 3208]"
6,"[HP:0001263, HP:0002540, HP:0001344, HP:000369...","[349, 542306, 2752, 813, 3208]"
7,"[HP:0000256, HP:0002126, HP:0410009, HP:001221...","[349, 2752, 542306, 813, 254361]"
8,"[HP:0000252, HP:0001252, HP:0001250]","[349, 2752, 542306, 254361, 352479]"
9,"[HP:0001266, HP:0001263, HP:0001321, HP:0011968]","[349, 813, 254361, 352479, 3208]"


In [18]:
df_results.to_csv('our_results.csv',index=False)